<a href="https://colab.research.google.com/github/beniamine3155/Fine_Tuning_LLM_with_HuggingFace/blob/main/LLM_Fine_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade transformers
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 21.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.


## Unsupervised Fine Tune / DAFT

In [2]:
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling
from transformers import AutoTokenizer, Trainer, TrainingArguments

In [3]:

corpus = [
    "Quantum computing is the future of technology.",
    "Artificial intelligence is transforming industries across the globe.",
    "Machine learning models improve with larger datasets and better features.",
    "Neural networks are the foundation of modern deep learning.",
    "The Large Hadron Collider was built to explore particle physics.",
    "Natural language processing enables machines to understand human speech.",
    "Data privacy is crucial in the age of big data.",
    "Edge computing reduces latency by processing data closer to the source.",
    "Blockchain technology enables decentralized trustless systems.",
    "The Turing Test measures a machine's ability to exhibit intelligent behavior.",
    "Augmented reality is being used in education, healthcare, and entertainment.",
    "Reinforcement learning allows agents to learn by trial and error.",
    "Genetic algorithms mimic natural selection for optimization problems.",
    "Speech recognition has advanced rapidly with transformer models.",
    "The quantum bit, or qubit, can exist in multiple states at once.",
    "Data lakes store raw, unstructured, and structured data.",
    "Autonomous vehicles rely heavily on computer vision systems.",
    "Cybersecurity threats are increasing with the rise of IoT devices.",
    "The metaverse could redefine how humans interact digitally.",
    "HPC (High Performance Computing) powers climate modeling and simulations.",
    "Supervised learning requires labeled datasets for training.",
    "Self-supervised learning is gaining popularity in NLP.",
    "Transfer learning reduces the amount of data needed for training.",
    "GPT models can generate human-like text based on prompts.",
    "Cloud-native architectures are scalable and resilient.",
    "Smart contracts execute automatically on blockchain platforms.",
    "Facial recognition systems have raised ethical concerns.",
    "The no-code movement democratizes app development.",
    "Differential privacy helps protect individual identities in datasets.",
    "Sustainable AI focuses on reducing energy consumption in training models.",
    "AI in healthcare is used for diagnostics, imaging, and personalized medicine.",
    "Bioinformatics uses computational tools to analyze biological data.",
    "Text embeddings map words into high-dimensional vector spaces.",
    "Zero-shot learning enables models to generalize to unseen tasks.",
    "TinyML enables machine learning on low-power microcontrollers.",
    "Explainable AI (XAI) makes model predictions more interpretable.",
    "GANs (Generative Adversarial Networks) generate realistic images and audio.",
    "Vision transformers are state-of-the-art for image classification.",
    "Federated learning allows models to train across decentralized data.",
    "Few-shot learning requires only a few examples to adapt.",
    "Deepfake technology raises questions about misinformation.",
    "Semantic search improves search accuracy using embeddings.",
    "Swarm robotics involves multiple robots working collaboratively.",
    "Prompt engineering improves model outputs in large language models.",
    "A/B testing helps validate product and UX changes.",
    "Synthetic data generation is useful for model training and privacy.",
    "Edge AI brings intelligence to mobile and embedded devices.",
    "Multimodal models can understand text, images, and audio together.",
    "Language models are trained using next-token prediction tasks."
]

In [4]:
from datasets import Dataset
text_dataset = Dataset.from_dict({"text":corpus})

In [5]:
text_dataset

Dataset({
    features: ['text'],
    num_rows: 49
})

In [7]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [8]:
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [9]:
model

DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0

In [10]:
def tokenize_corpus(example):
  return tokenizer(example["text"], truncation=True, padding = "max_length",  max_length=64)


tokenized_corpus = text_dataset.map(tokenize_corpus)

Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [15]:
args = TrainingArguments(
    output_dir="./results_uft",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    logging_strategy="no",
    save_strategy="no",
    report_to="none"
)

In [16]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = tokenized_corpus,
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)
)

In [17]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=70, training_loss=2.010985892159598, metrics={'train_runtime': 5.112, 'train_samples_per_second': 95.854, 'train_steps_per_second': 13.693, 'total_flos': 8119370810880.0, 'train_loss': 2.010985892159598, 'epoch': 10.0})

In [18]:
# Save model and tokenizer to a directory
save_path = "./unsupervised_fine_tune"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./unsupervised_fine_tune/tokenizer_config.json',
 './unsupervised_fine_tune/special_tokens_map.json',
 './unsupervised_fine_tune/vocab.txt',
 './unsupervised_fine_tune/added_tokens.json',
 './unsupervised_fine_tune/tokenizer.json')

In [19]:
# Load the Fine Tune Model
from transformers import AutoModelForCausalLM, AutoTokenizer
load_path = "./unsupervised_fine_tune"
tokenizer = AutoTokenizer.from_pretrained(load_path)
model = AutoModelForMaskedLM.from_pretrained(load_path)

In [23]:
import torch

# Example of a sentence with a mask token
text = "GANs (Generative Adversarial Networks) generate [MASK] images and audio."
# Tokenize the input
inputs = tokenizer(text, return_tensors="pt")

# Check if [MASK] is in the input
if tokenizer.mask_token_id not in inputs["input_ids"]:
    raise ValueError("No [MASK] token found in the input.")


# Predict
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get index of [MASK] token
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

# Get top 5 predictions for [MASK]
top_k = 5
mask_logits = logits[0, mask_token_index, :]
top_tokens = torch.topk(mask_logits, top_k, dim=1).indices[0].tolist()

# Display predictions
print("Top predictions for [MASK]:")
for token_id in top_tokens:
    print(tokenizer.decode([token_id]))

Top predictions for [MASK]:
digital
both
video
visual
together


## Supervised Fine Tune

In [34]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [35]:
dataset = load_dataset("imdb")
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(2000))
small_test_dataset = dataset["test"].shuffle(seed=42).select(range(500))

In [36]:
small_train_dataset.shape, small_test_dataset.shape

((2000, 2), (500, 2))

In [37]:
# Load Model and Tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:

# Tokenize
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

tokenized_train = small_train_dataset.map(tokenize, batched=True)
tokenized_test = small_test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [39]:
df = small_test_dataset.to_pandas()
df.head()

,text,label
0,<br /><br />When I unsuspectedly rented A Thou...,1
1,This is the latest entry in the long series of...,1
2,This movie was so frustrating. Everything seem...,0
3,"I was truly and wonderfully surprised at ""O' B...",1
4,This movie spends most of its time preaching t...,0


In [41]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    save_strategy="no",
    logging_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=2500, training_loss=0.0679306396484375, metrics={'train_runtime': 930.9392, 'train_samples_per_second': 21.484, 'train_steps_per_second': 2.685, 'total_flos': 2649347973120000.0, 'train_loss': 0.0679306396484375, 'epoch': 10.0})

In [42]:
# Save model and tokenizer to a directory
save_path = "./supervised_fine_tune"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./supervised_fine_tune/tokenizer_config.json',
 './supervised_fine_tune/special_tokens_map.json',
 './supervised_fine_tune/vocab.txt',
 './supervised_fine_tune/added_tokens.json',
 './supervised_fine_tune/tokenizer.json')

In [43]:
load_path = "./supervised_fine_tune"
tokenizer = AutoTokenizer.from_pretrained(load_path)
model = AutoModelForMaskedLM.from_pretrained(load_path)

Some weights of DistilBertForMaskedLM were not initialized from the model checkpoint at ./supervised_fine_tune and are newly initialized: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
# Evaluate model performance
metrics = trainer.evaluate()
print("Evaluation metrics:", metrics)

Evaluation metrics: {'eval_loss': 0.8027961254119873, 'eval_runtime': 7.0806, 'eval_samples_per_second': 70.616, 'eval_steps_per_second': 8.898, 'epoch': 10.0}


In [47]:
text = "The movie was fantastic, I really enjoyed it!"

# Tokenize input
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=256)

# Load the supervised fine-tuned model
load_path = "./supervised_fine_tune"
model = AutoModelForSequenceClassification.from_pretrained(load_path)

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Map prediction to label
labels = ["Negative", "Positive"]
print(f"Text: {text}")
print(f"Predicted Sentiment: {labels[predicted_class]}")

Text: The movie was fantastic, I really enjoyed it!
Predicted Sentiment: Positive


## Instruction based Fine Tune

In [48]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling


In [49]:
# Use Alpaca-style dataset structure (mocked here)
data = [
    {"instruction": "Translate to French", "input": "Good morning", "output": "Bonjour"},
    {"instruction": "Translate to French", "input": "How are you?", "output": "Comment ça va ?"},
    {"instruction": "Translate to French", "input": "I am learning AI", "output": "J'apprends l'intelligence artificielle"},
    {"instruction": "Translate to French", "input": "What is your name?", "output": "Comment tu t'appelles ?"},
    {"instruction": "Translate to French", "input": "Thank you very much", "output": "Merci beaucoup"},
    {"instruction": "Translate to French", "input": "I love programming", "output": "J'adore programmer"},
    {"instruction": "Translate to French", "input": "Where is the train station?", "output": "Où est la gare ?"},
    {"instruction": "Translate to French", "input": "This is my book", "output": "C'est mon livre"},
    {"instruction": "Translate to French", "input": "See you tomorrow", "output": "À demain"},
    {"instruction": "Translate to French", "input": "She is a doctor", "output": "Elle est médecin"},
    {"instruction": "Translate to French", "input": "What time is it?", "output": "Quelle heure est-il ?"},
    {"instruction": "Translate to French", "input": "I am very tired", "output": "Je suis très fatigué"},
    {"instruction": "Translate to French", "input": "Can you help me?", "output": "Pouvez-vous m'aider ?"},
    {"instruction": "Translate to French", "input": "I like your idea", "output": "J'aime ton idée"},
    {"instruction": "Translate to French", "input": "Please sit down", "output": "Veuillez vous asseoir"},
    {"instruction": "Translate to French", "input": "Do you speak English?", "output": "Parlez-vous anglais ?"},
    {"instruction": "Translate to French", "input": "It is raining today", "output": "Il pleut aujourd'hui"},
    {"instruction": "Translate to French", "input": "I am from Bangladesh", "output": "Je viens du Bangladesh"},
    {"instruction": "Translate to French", "input": "He is my best friend", "output": "C'est mon meilleur ami"},
    {"instruction": "Translate to French", "input": "Where do you live?", "output": "Où habites-tu ?"},
    {"instruction": "Translate to French", "input": "I am feeling hungry", "output": "J'ai faim"},
    {"instruction": "Translate to French", "input": "Can I have some water?", "output": "Puis-je avoir de l'eau ?"},
    {"instruction": "Translate to French", "input": "My favorite color is blue", "output": "Ma couleur préférée est le bleu"},
    {"instruction": "Translate to French", "input": "I am watching a movie", "output": "Je regarde un film"},
    {"instruction": "Translate to French", "input": "They are playing football", "output": "Ils jouent au football"},
    {"instruction": "Translate to French", "input": "Do you like music?", "output": "Aimes-tu la musique ?"},
    {"instruction": "Translate to French", "input": "It’s a beautiful day", "output": "C'est une belle journée"},
    {"instruction": "Translate to French", "input": "Where is the bathroom?", "output": "Où sont les toilettes ?"},
    {"instruction": "Translate to French", "input": "I am learning Python", "output": "J'apprends le Python"},
    {"instruction": "Translate to French", "input": "This food tastes great", "output": "Ce plat est délicieux"},
    {"instruction": "Translate to French", "input": "Can I pay by card?", "output": "Puis-je payer par carte ?"},
    {"instruction": "Translate to French", "input": "I have two sisters", "output": "J'ai deux sœurs"},
    {"instruction": "Translate to French", "input": "Where are you going?", "output": "Où vas-tu ?"},
    {"instruction": "Translate to French", "input": "He is reading a book", "output": "Il lit un livre"},
    {"instruction": "Translate to French", "input": "I don’t understand", "output": "Je ne comprends pas"},
    {"instruction": "Translate to French", "input": "I need some rest", "output": "J'ai besoin de repos"},
    {"instruction": "Translate to French", "input": "Open the window please", "output": "Ouvre la fenêtre s'il te plaît"},
    {"instruction": "Translate to French", "input": "I like to travel", "output": "J'aime voyager"},
    {"instruction": "Translate to French", "input": "How old are you?", "output": "Quel âge as-tu ?"},
    {"instruction": "Translate to French", "input": "Where is my phone?", "output": "Où est mon téléphone ?"},
    {"instruction": "Translate to French", "input": "I am going to school", "output": "Je vais à l'école"},
    {"instruction": "Translate to French", "input": "Turn off the lights", "output": "Éteins les lumières"},
    {"instruction": "Translate to French", "input": "I will call you later", "output": "Je t'appellerai plus tard"},
    {"instruction": "Translate to French", "input": "He is watching TV", "output": "Il regarde la télévision"},
    {"instruction": "Translate to French", "input": "I am cooking dinner", "output": "Je prépare le dîner"},
    {"instruction": "Translate to French", "input": "What’s your favorite movie?", "output": "Quel est ton film préféré ?"},
    {"instruction": "Translate to French", "input": "I lost my keys", "output": "J'ai perdu mes clés"},
    {"instruction": "Translate to French", "input": "I will be there soon", "output": "J'arriverai bientôt"},
    {"instruction": "Translate to French", "input": "It’s too expensive", "output": "C'est trop cher"},
    {"instruction": "Translate to French", "input": "Happy birthday!", "output": "Joyeux anniversaire !"},
    {"instruction": "Translate to French", "input": "I am feeling cold", "output": "J'ai froid"},
]


In [50]:
from datasets import Dataset
instruction_dataset = Dataset.from_list(data)

In [51]:
instruction_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 51
})

In [52]:
model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [53]:
def format_and_tokenize(example):
    prompt = (
        f"### Instruction:\n{example['instruction']}\n"
        f"### Input:\n{example['input']}\n"
        f"### Response:\n{example['output']}"
    )
    return tokenizer(prompt, padding="max_length", truncation=True, max_length=128)


tokenized_data = instruction_dataset.map(format_and_tokenize)

Map:   0%|          | 0/51 [00:00<?, ? examples/s]

In [54]:
args = TrainingArguments(
    output_dir="./results_instruction",
    per_device_train_batch_size=4,
    num_train_epochs=15,
    logging_strategy="no",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_data,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)

trainer.train()

Step,Training Loss


TrainOutput(global_step=195, training_loss=0.34802469106820916, metrics={'train_runtime': 30.568, 'train_samples_per_second': 25.026, 'train_steps_per_second': 6.379, 'total_flos': 49955857367040.0, 'train_loss': 0.34802469106820916, 'epoch': 15.0})

In [55]:
# Save and Load Model
trainer.save_model("./results_instruction")
tokenizer.save_pretrained("./results_instruction")

# Reload
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("./results_instruction")
tokenizer = AutoTokenizer.from_pretrained("./results_instruction")

In [56]:
prompt = """### Instruction:
Translate to French
### Input:
Where is the train station?
### Response:"""

inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(
    **inputs,
    max_length=100,
    do_sample=False,
    num_beams=3,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Extract clean output
response = generated_text.split("### Response:")[-1].strip().split("\n")[0]
print("Model response:", response)

Model response: Où est la gare?
